<a href="https://colab.research.google.com/github/Lakshaykumarr28/Machine-Learning-using-Python/blob/main/DL/ResNet_CNN_model_for_Caltech101_Lakshay_Kumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Lakshaykumarr28/Machine-Learning-using-Python

fatal: destination path 'Machine-Learning-using-Python' already exists and is not an empty directory.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Sequential


In [ ]:
data_dir = '/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to match ResNet's input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 7356 images belonging to 102 classes.
Found 1788 images belonging to 102 classes.


In [ ]:
import cv2
import numpy as np
import os
#------------
def normalize_image(image_path, img_shape):
    read_image = cv2.imread(image_path)

    print(image_path)

    image_resized = cv2.resize(read_image, img_shape, interpolation=cv2.INTER_CUBIC)

    image = np.float32(image_resized)
    image = cv2.normalize(image, image, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX) #Change alpha, beta according to the preprocessing you desire

    return image

def getLabel(input_string):
    try:
        i=input_string.index('-101')
        return input_string[4:i]
    except ValueError:
        return input_string[4:]

from sklearn.model_selection import train_test_split
#-------------------
def get_images_and_classes(main_path, partition, img_shape=(128, 128)):

    images_training = []
    y_training = []
    images_testing = []
    y_testing = []
    list = [name for name in os.listdir(main_path) if os.path.isdir(os.path.join(main_path,name))]
    for idx,folder in enumerate(list):
        label = getLabel(folder)
        sub_list = sorted(os.listdir(os.path.join(main_path,folder)))

        local_y= []
        local_images= []
        for i in range(0, len(sub_list)):
            image_path = os.path.join(main_path, folder, sub_list[i])
            image= normalize_image(image_path, img_shape)

            local_images.append(image)
            local_y.append(label)

        X_train, X_test, y_train, y_test = train_test_split(local_images, local_y, test_size = partition)

        images_training = images_training + X_train
        y_training = y_training + y_train
        images_testing = images_testing + X_test
        y_testing = y_testing + y_test

    images_training = np.array(images_training)
    y_training = np.array(y_training)
    images_testing = np.array(images_testing)
    y_testing = np.array(y_testing)
    return images_training, y_training, images_testing, y_testing

In [ ]:
path='/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories'
partition= 0.2
images_training, y_training, images_testing, y_testing = get_images_and_classes(path, partition)

Streaming output truncated to the last 5000 lines.
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0006.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0007.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0008.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0009.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0010.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0011.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0012.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0013.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cellphone/image_0014.jpg
/content/Machine-Learning-using-Python/caltech-101/101_ObjectCategories/cell

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

batch_size = 32
img_size = (224, 224)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
y_training = onehotencoder.fit_transform(y_training.reshape(-1, 1)).toarray()
y_testing = onehotencoder.transform(y_testing.reshape(-1, 1)).toarray()

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(train_generator.num_classes, activation='softmax')
])

# Freeze the layers of the pre-trained ResNet model
for layer in base_model.layers:
    layer.trainable = False


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 3

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/3
229/229 [==============================] - 1869s 8s/step - loss: 3.5831 - accuracy: 0.2544 - val_loss: 5.8376 - val_accuracy: 0.0585
Epoch 2/3
229/229 [==============================] - 1859s 8s/step - loss: 3.0693 - accuracy: 0.3281 - val_loss: 5.6254 - val_accuracy: 0.1756
Epoch 3/3
229/229 [==============================] - 1848s 8s/step - loss: 2.8255 - accuracy: 0.3617 - val_loss: 4.5270 - val_accuracy: 0.1295


In [ ]:
from torch.utils.data import DataLoader, Dataset
class ImageDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms

    def __len__(self):
        return (len(self.X))

    def __getitem__(self, i):
        data = self.X[i][:]

        if self.transforms:
            data = self.transforms(data)

        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [ ]:
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important to keep predictions matched with true labels
)

Found 9144 images belonging to 102 classes.


In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

56/56 [==============================] - 334s 6s/step - loss: 4.4751 - accuracy: 0.1342
Validation Loss: 4.47507381439209
Validation Accuracy: 0.1342281848192215


In [ ]:
# Get the class names
class_names = sorted(train_generator.class_indices, key=lambda k: train_generator.class_indices[k])

# Predict using the model
predictions = model.predict(test_generator)

# Convert predictions to class names
predicted_classes = np.argmax(predictions, axis=1)
predicted_class_names = [class_names[i] for i in predicted_classes]

# Print testing predictions
for filename, predicted_class in zip(test_generator.filenames, predicted_class_names):
    print(f"File: {filename} - Predicted Class: {predicted_class}")

Streaming output truncated to the last 5000 lines.
File: ceiling_fan/image_0030.jpg - Predicted Class: menorah
File: ceiling_fan/image_0031.jpg - Predicted Class: menorah
File: ceiling_fan/image_0032.jpg - Predicted Class: inline_skate
File: ceiling_fan/image_0033.jpg - Predicted Class: Faces_easy
File: ceiling_fan/image_0034.jpg - Predicted Class: menorah
File: ceiling_fan/image_0035.jpg - Predicted Class: menorah
File: ceiling_fan/image_0036.jpg - Predicted Class: menorah
File: ceiling_fan/image_0037.jpg - Predicted Class: menorah
File: ceiling_fan/image_0038.jpg - Predicted Class: menorah
File: ceiling_fan/image_0039.jpg - Predicted Class: menorah
File: ceiling_fan/image_0040.jpg - Predicted Class: menorah
File: ceiling_fan/image_0041.jpg - Predicted Class: menorah
File: ceiling_fan/image_0042.jpg - Predicted Class: watch
File: ceiling_fan/image_0043.jpg - Predicted Class: menorah
File: ceiling_fan/image_0044.jpg - Predicted Class: menorah
File: ceiling_fan/image_0045.jpg - Predicte

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

286/286 [==============================] - 1610s 6s/step - loss: 3.7916 - accuracy: 0.2093
Test Loss: 3.7916321754455566
Test Accuracy: 0.20931757986545563
